In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from collections import defaultdict
import gradio as gr
import matplotlib.pyplot as plt
import os

# Load YOLOv8 model
yolo_model = YOLO("C:/Users/PERCY JACKSON/OneDrive/Desktop/best.pt")

# Load InceptionResNetV2 classification model
inception_model = load_model("C:/Users/PERCY JACKSON/OneDrive/Desktop/Restnetv2_model.keras")

# Class ID to Name mapping
class_names = {
    0: 'Cardboard',
    1: 'Glass',
    2: 'Metal',
    3: 'Paper',
    4: 'Plastic',
    5: 'Trash'
}






C:\Users\PERCY JACKSON\AppData\Roaming\Python\Python312\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
def process_image(image):
    # Save temporarily
    temp_path = "C:/Users/PERCY JACKSON/OneDrive/Desktop/test2.jpg"
    cv2.imwrite(temp_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

    # Step 1: Detection
    results = yolo_model(temp_path, conf=0.5, iou=0.3)
    detected_boxes = []
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            confidence = float(box.conf[0])
            class_id = int(box.cls[0])
            detected_boxes.append((x1, y1, x2, y2, confidence, class_id))

    # Step 2: Cropping and preprocessing
    cropped_images = []
    img = cv2.imread(temp_path)
    for (x1, y1, x2, y2, confidence, class_id) in detected_boxes:
        cropped_img = img[y1:y2, x1:x2]
        if cropped_img.size == 0:
            continue
        resized_img = cv2.resize(cropped_img, (224, 224)) / 255.0
        cropped_images.append((resized_img, (x1, y1, x2, y2, class_id)))

    # Step 3: Classify with Inception model
    corrected_labels = []
    for img, (x1, y1, x2, y2, class_id) in cropped_images:
        img_input = np.expand_dims(img, axis=0)
        prediction = inception_model.predict(img_input, verbose=0)
        corrected_class_id = np.argmax(prediction)
        corrected_labels.append((x1, y1, x2, y2, corrected_class_id))

    # Step 4: Draw final results
    final_img = cv2.imread(temp_path)
    final_img = cv2.cvtColor(final_img, cv2.COLOR_BGR2RGB)
    for (x1, y1, x2, y2, class_id) in corrected_labels:
        class_name = class_names.get(class_id, f"Class {class_id}")
        cv2.rectangle(final_img, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(final_img, f"{class_name}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    # Step 5: Percentage calculation
    class_areas = defaultdict(float)
    total_area = 0.0
    for (x1, y1, x2, y2, class_id) in corrected_labels:
        area = (x2 - x1) * (y2 - y1)
        class_areas[class_id] += area
        total_area += area

    percentages = {class_names[class_id]: round((area / total_area) * 100, 2)
                   for class_id, area in class_areas.items()}

    # Step 6: Generate label output
    percentage_str = "\n".join([f"{k}: {v}%" for k, v in percentages.items()])

    return final_img, percentage_str

In [4]:
# Gradio interface
interface = gr.Interface(
    fn=process_image,
    inputs=gr.Image(type="numpy", label="Upload Waste Image"),
    outputs=[
        gr.Image(type="numpy", label="Detected Output"),
        gr.Textbox(label="Class Percentage Composition")
    ],
    title="Smart Waste Classifier",
    description="Upload an image to detect and classify waste items using YOLOv8 + InceptionResNetV2 pipeline."
)

interface.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.



image 1/1 C:\Users\PERCY JACKSON\OneDrive\Desktop\test2.jpg: 640x480 1 Cardboard, 1 Glass, 2 Metals, 1 Paper, 1 Trash, 352.1ms
Speed: 16.0ms preprocess, 352.1ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 480)
